In [1]:
import torch
import Models.misc as misc
import Inference.Kolmogorov.performance as performance
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Emu_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250526_223850-r12kgbg1/files/checkpoint_best.p'
CT_file_path = '/scratch/ql2221/thermalizer_data/wandb_data/wandb/run-20250715_210115-naiteaau/files/checkpoint_last.p'

CT = misc.load_diffusion_model(CT_file_path).to(device)
Emu = misc.load_model(Emu_file_path).to(device)

In [2]:
data_dict = torch.load("/scratch/ql2221/thermalizer_data/kolmogorov/reynold10k/test_long.p")

In [3]:
data = data_dict["data"]
print(data.shape)
x = data[0,:,:,:]
x = x.unsqueeze(0).to(device)
print(x.shape)
x = x/4.44

torch.Size([5, 70000, 64, 64])
torch.Size([1, 70000, 64, 64])


In [4]:
x_t_noised = x[:,1:2]
x_t_minus = x[:,0:1]

noised_plus_conditional= torch.cat((x_t_noised, x_t_minus), dim=1)
delta = torch.tensor([1]).to(device)
pred_noise, pred_noise_level = CT.model(noised_plus_conditional,delta,True)
pred_noise_level


tensor([[ -3.6913,  -5.1007,  -4.6001,  -4.6313,  -3.7631,  -4.1202,  -4.8398,
          -5.0625,  -5.1515,  -5.0992,  -5.1920,  -5.1410,  -5.0732,  -4.5911,
          -5.0131,  -4.7349,  -4.8674,  -4.5680,  -4.6539,  -4.1613,  -3.9007,
          -3.9530,  -3.6546,  -3.4271,  -3.8797,  -3.9165,  -3.8641,  -3.9769,
          -4.3395,  -3.8093,  -4.2888,  -4.0805,  -4.3550,  -3.9646,  -4.5163,
          -4.1582,  -3.7178,  -3.5578,  -3.2317,  -2.7861,  -2.9309,  -2.5412,
          -2.0137,  -1.8975,  -2.2074,  -1.7683,  -1.5215,  -1.3681,  -1.5880,
          -1.4596,  -1.6486,  -1.7498,  -1.6523,  -1.9130,  -1.7468,  -1.7711,
          -1.9896,  -1.9999,  -2.2537,  -2.5596,  -2.4049,  -2.3993,  -2.5670,
          -3.0325,  -3.2430,  -2.6593,  -2.4782,  -2.9740,  -2.9751,  -3.3066,
          -3.2090,  -2.8442,  -3.1264,  -2.7329,  -2.7533,  -2.4101,  -2.8307,
          -2.3625,  -2.6928,  -2.0333,  -2.4297,  -2.3730,  -2.4574,  -2.3911,
          -2.1845,  -2.7329,  -2.5440,  -2.3051,  -2

In [6]:
delta = torch.tensor([1]).to(device)
rollout, _ = performance.run_conditional_emu(x[:,0:1], Emu, CT, n_steps=1000, lag = delta, denoising_steps=5, freq = 25, silent=True, sigma=None, Regression = True)

AttributeError: 'int' object has no attribute 'shape'